pip install tavily-python huggingface-hub langgraph

importing and fetcheing the api key

In [20]:
from tavily import TavilyClient
from huggingface_hub import InferenceClient
from langgraph.graph import StateGraph
from typing import TypedDict
from dotenv import load_dotenv
import os

load_dotenv()

client = TavilyClient(os.environ.get("TAVILY_API_KEY"))
hf_client = InferenceClient(..., token=os.environ.get("HF_API_TOKEN"))

Variables

In [21]:
class ResearchState(TypedDict):
    query: str
    research_data: str
    final_answer: str

function for web crawler

In [22]:
def research_agent(state):
    """Fetches relevant research data from Tavily and returns structured results."""
    if not isinstance(state, dict):
        raise TypeError(f"Expected dictionary, but got {type(state)}")
    if "query" not in state:
        raise KeyError("State does not contain 'query' key.")
    query = state["query"]
    print("Researching query:", query) 

# Fetch research data
    response = client.search(query=query, max_results=5)

# Ensure response is a dictionary
    if not isinstance(response, dict):
        raise TypeError(f"Unexpected research data type: {type(response)}")

# Extract the actual search results
    research_results = response.get("results", [])

    if not isinstance(research_results, list):  # Ensure it's a list
        raise TypeError(f"Expected 'results' to be a list, but got {type(research_results)}")

    if len(research_results) == 0:
        print("No research data found.")
        return {"query": query, "research_data": [], "final_answer": ""}
# extreacting related filds
    structured_data = [
        {
            "title": item.get("title", "No Title"),
            "url": item.get("url", "No URL"),
            "content": item.get("content", "No Content")[:500],  # Limit content length
            "score": item.get("score", 0)
        }
        for item in research_results
    ]
    # print("Data Collected:", structured_data)
    return {"query": query, "research_data": structured_data, "final_answer": ""}


organizing the data by providing the fetched information from the browser to ai

In [23]:
def answer_drafter_agent(state: ResearchState) -> ResearchState:
    """Summarizes research data and generates a structured answer."""
    research_data = state["research_data"]

    if not research_data:
        return {"query": state["query"], "research_data": research_data, "final_answer": "No data available."}
    try:
        key_points = "\n".join([f"- {item['title']}: {item['content'][:200]}..." for item in research_data[:3]])  #only three results
        prompt = f"Based on the following research findings, write a concise, structured summary of the latest AI advancements in cybersecurity:\n{key_points}"
        response = hf_client.text_generation(prompt, max_new_tokens=500, temperature=0.7)
        return {"query": state["query"], "research_data": research_data, "final_answer": response}
    except Exception as e:
        return {"query": state["query"], "research_data": research_data, "final_answer": f"Error: {e}"}


Creating the Multi agent system 

In [24]:
graph = StateGraph(ResearchState)  # Pass state schema
# Add nodes for each agent
graph.add_node("research_agent", research_agent)
graph.add_node("answer_drafter_agent", answer_drafter_agent)
# Define edges (data flow between agents)
graph.add_edge("research_agent", "answer_drafter_agent")
# Set entry point (where execution starts)
graph.set_entry_point("research_agent")
executor = graph.compile()

Example

In [25]:
query_input = "Latest advancements in AI for cybersecurity"

initial_state = {"query": query_input, "research_data": "", "final_answer": ""}
final_state = executor.invoke(initial_state)

print("\n🔍 Research Data:\n", final_state["research_data"])
print("\n📝 Final Structured Answer:\n", final_state["final_answer"])


Researching query: Latest advancements in AI for cybersecurity

🔍 Research Data:
 [{'title': 'AI and Cyber Security: Innovations & Challenges', 'url': 'https://www.esecurityplanet.com/trends/ai-and-cybersecurity-innovations-and-challenges/', 'content': 'Artificial intelligence in cybersecurity refers to applying AI technologies such as machine learning, deep learning, and data analytics to protect digital systems and networks from cyber threats. By identifying deviations from normal patterns, AI can detect potential security incidents before they escalate into serious threats. Security Information and Event Management (SIEM): AI advances SIEM by providing more accurate insights into security events and incidents, helping organizations respond s', 'score': 0.98583}, {'title': 'AI-driven fusion with cybersecurity: Exploring current trends, advanced ...', 'url': 'https://www.sciencedirect.com/science/article/pii/S1566253524007000', 'content': 'This comprehensive review explores current AI

as Open ai and other good AI are not free and i searched for other AI to analyse the data so i got a free one and i used it but it dosent provide a proper analyzed answer as its limited to few words so this is all for now i can even do some alteration so that it can generate good analyzed data.
(Also some times the server dosent respond mainly from falcon - hugging face)